In [1]:
!git clone https://github.com/nhoues/Water-Level-Forecasting.git

Cloning into 'Water-Level-Forecasting'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 210 (delta 110), reused 182 (delta 85), pack-reused 0
Receiving objects: 100% (210/210), 15.89 MiB | 15.69 MiB/s, done.
Resolving deltas: 100% (110/110), done.


In [2]:
cd Water-Level-Forecasting 

/content/Water-Level-Forecasting


# Set Up 

In [3]:
import os 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

# Load the Hydrometric stations Data 

In [4]:
data_dir = 'data/hydro/'
all_data_paths = os.listdir(data_dir)

In [5]:
def add_nans(x): 
    if x == 'nan' : 
        return np.nan 
    else : 
        return x 

In [6]:
def corrcet_id(x) : 
    if x[0] == '\ufeff' : 
        x = x[1:]
    return x 

In [7]:
all_data = []
all_ids = []
for file_path in all_data_paths : 
    if '.csv' in file_path : 
        data = pd.read_csv(data_dir + file_path )
        data = pd.DataFrame( columns = data.index[0] , data = np.vstack(data.index[1:]))
        data[' ID'] = data[' ID'].apply(corrcet_id)
        for id_ in data[' ID'].unique() : 
            if id_ not in all_ids : 
                all_ids.append(id_)
                temp = data[data[' ID'] == id_].copy()
                datetime_series = pd.to_datetime(temp['Date'])
                datetime_index = pd.DatetimeIndex(datetime_series.values)
                temp.set_index(datetime_index,inplace =True)
                temp.drop('Date',1,inplace =True)
                temp = temp[temp.index > '2000-01-01']
                temp['Value'] =  temp['Value'].apply(add_nans) 
                pct = temp['Value'].isnull().sum()/temp.shape[0]
                if pct < 0.1 : 
                    all_data.append(temp)
all_data = pd.concat(all_data)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in long_scalars


In [8]:
all_data.head()

,ID,PARAM,Value
2000-01-02,07EA007,1,5.18
2000-01-03,07EA007,1,5.15
2000-01-04,07EA007,1,5.10
2000-01-05,07EA007,1,5.04
2000-01-06,07EA007,1,4.95


### Adding Latitude	and Longitude

In [9]:
stations_list = pd.read_csv('data/hydrometric_StationList (5).csv') 

In [10]:
def corrcet_id(x) : 
    if x[0] == '\ufeff' : 
        x = x[1:]
    return x 
stations_list[' ID'] = stations_list[' ID'].apply(corrcet_id) 

In [11]:
all_data.reset_index(inplace = True )  

In [12]:
all_data = pd.merge(all_data , stations_list[[' ID' ,'Latitude', 'Longitude']] , on= ' ID' , how = 'left') 


In [13]:
all_data

,index,ID,PARAM,Value,Latitude,Longitude
0,2000-01-02,07EA007,1,5.18,57.19111,-124.90111
1,2000-01-03,07EA007,1,5.15,57.19111,-124.90111
2,2000-01-04,07EA007,1,5.10,57.19111,-124.90111
3,2000-01-05,07EA007,1,5.04,57.19111,-124.90111
4,2000-01-06,07EA007,1,4.95,57.19111,-124.90111
...,...,...,...,...,...,...
579246,2019-12-27,08MG012,2,9.019,49.30926,-121.77373
579247,2019-12-28,08MG012,2,8.998,49.30926,-121.77373
579248,2019-12-29,08MG012,2,8.975,49.30926,-121.77373
579249,2019-12-30,08MG012,2,8.946,49.30926,-121.77373


In [14]:
all_data.drop(all_data[all_data['Longitude'].isnull()].index , inplace = True)
all_data.head()

,index,ID,PARAM,Value,Latitude,Longitude
0,2000-01-02,07EA007,1,5.18,57.19111,-124.90111
1,2000-01-03,07EA007,1,5.15,57.19111,-124.90111
2,2000-01-04,07EA007,1,5.10,57.19111,-124.90111
3,2000-01-05,07EA007,1,5.04,57.19111,-124.90111
4,2000-01-06,07EA007,1,4.95,57.19111,-124.90111


In [15]:
print('number of staions =' , all_data[' ID'].nunique() )

number of staions = 62


### Filling missed Values 

In [16]:
print('Missing value ratio : ') 
all_data.isnull().sum() / all_data.shape[0]

Missing value ratio : 


index        0.000000
 ID          0.000000
PARAM        0.000000
Value        0.016191
Latitude     0.000000
Longitude    0.000000
dtype: float64

In [19]:
all_data.fillna(method='ffill' , inplace = True)

In [20]:
all_data.columns = ['Date', 'ID', 'PARAM', 'Value', 'Latitude', 'Longitude']

In [21]:
all_data.to_csv('preprocessed_data.csv' , index = False )

In [22]:
all_data.ID.unique()

array(['07EA007', '05TG005', '02YF002', '05KL005', '05AB037', '02OH001',
       '08MH149', '02HB021', '02JD013', '05LK002', '05LL012', '02DD006',
       '05MD009', '05PD029', '05PE014', '05LM005', '02JE011', '05RD005',
       '05KK002', '02YL009', '05UD001', '05LJ009', '02YL007', '08NH127',
       '06EB007', '05RD006', '05TF001', '01AO010', '05CB006', '01AR013',
       '01AR009', '01AO003', '02YK010', '05ND012', '08LG046', '05KK005',
       '05PF070', '08NM083', '05AA032', '02JE024', '02LA015', '02OA013',
       '08FA007', '05QC004', '05UB005', '06EA012', '08KH011', '05NB032',
       '05PB024', '05PB007', '05TF004', '05RE003', '05SB006', '05SD002',
       '05SG001', '05RF001', '05SD001', '05SA003', '05LH001', '02GA030',
       '06EB002', '08MG012'], dtype=object)

# Add Climate Data 

In [53]:
target_station = '08MG012'
data = all_data[all_data['ID']==target_station].copy()
data['Date'] =pd.to_datetime(data['Date']) 
data.set_index('Date' , inplace = True) 
data.sort_index(inplace = True)

In [54]:
path_to_climate_data = 'data/climate'
climate_data = []
for path in os.listdir(path_to_climate_data) :
  climate_data.append(pd.read_csv(path_to_climate_data + '/' + path)) 
climate_data = pd.concat(climate_data) 
climate_data.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,Min Temp (°C),Min Temp Flag,Mean Temp (°C),Mean Temp Flag,Heat Deg Days (°C),Heat Deg Days Flag,Cool Deg Days (°C),Cool Deg Days Flag,Total Rain (mm),Total Rain Flag,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,-121.76,49.24,AGASSIZ RCS,1100119,2011-01-01,2011,1,1,NaN,0.4,NaN,-3.0,NaN,-1.3,NaN,19.3,NaN,0.0,NaN,NaN,M,NaN,M,0.0,NaN,NaN,NaN,2.0,NaN,54,NaN
1,-121.76,49.24,AGASSIZ RCS,1100119,2011-01-02,2011,1,2,NaN,3.1,NaN,-3.0,NaN,0.1,NaN,17.9,NaN,0.0,NaN,NaN,M,NaN,M,0.4,NaN,NaN,NaN,4.0,NaN,56,NaN
2,-121.76,49.24,AGASSIZ RCS,1100119,2011-01-03,2011,1,3,NaN,5.4,NaN,-2.3,NaN,1.6,NaN,16.4,NaN,0.0,NaN,NaN,M,NaN,M,0.6,NaN,NaN,NaN,2.0,NaN,50,NaN
3,-121.76,49.24,AGASSIZ RCS,1100119,2011-01-04,2011,1,4,NaN,0.9,NaN,-4.1,NaN,-1.6,NaN,19.6,NaN,0.0,NaN,NaN,M,NaN,M,0.2,NaN,NaN,NaN,NaN,NaN,<31,NaN
4,-121.76,49.24,AGASSIZ RCS,1100119,2011-01-05,2011,1,5,NaN,1.0,NaN,-0.4,NaN,0.3,NaN,17.7,NaN,0.0,NaN,NaN,M,NaN,M,26.2,NaN,NaN,NaN,NaN,NaN,<31,NaN


In [55]:
climate_data.isnull().mean()

Longitude (x)                0.000000
Latitude (y)                 0.000000
Station Name                 0.000000
Climate ID                   0.000000
Date/Time                    0.000000
Year                         0.000000
Month                        0.000000
Day                          0.000000
Data Quality                 1.000000
Max Temp (°C)                0.023986
Max Temp Flag                0.981749
Min Temp (°C)                0.013949
Min Temp Flag                0.984748
Mean Temp (°C)               0.024899
Mean Temp Flag               0.980446
Heat Deg Days (°C)           0.024899
Heat Deg Days Flag           0.980446
Cool Deg Days (°C)           0.024899
Cool Deg Days Flag           0.980446
Total Rain (mm)              1.000000
Total Rain Flag              0.466953
Total Snow (cm)              1.000000
Total Snow Flag              0.467084
Total Precip (mm)            0.086038
Total Precip Flag            0.938209
Snow on Grnd (cm)            0.970538
Snow on Grnd

In [56]:
climate_data = climate_data[['Date/Time','Max Temp (°C)','Min Temp (°C)','Mean Temp (°C)','Heat Deg Days (°C)','Cool Deg Days (°C)','Total Precip (mm)']]
climate_data.columns = ['Date','Max Temp','Min Temp','Mean Temp','Heat Deg Days','Cool Deg Days','Total Precip']
climate_data['Date'] = pd.to_datetime(climate_data['Date']) 
climate_data.set_index('Date' , inplace =True ) 
climate_data.sort_index(inplace = True)
climate_data.fillna(method='ffill' , inplace = True)
climate_data.head()

,Max Temp,Min Temp,Mean Temp,Heat Deg Days,Cool Deg Days,Total Precip
Date,,,,,,
2000-01-01,5.4,2.3,3.9,14.1,0.0,24.2
2000-01-02,5.3,-1.3,2.0,16.0,0.0,5.0
2000-01-03,2.2,-1.1,0.6,17.4,0.0,9.0
2000-01-04,9.1,0.9,5.0,13.0,0.0,10.8
2000-01-05,7.5,1.6,4.6,13.4,0.0,0.0


In [57]:
for feat in climate_data.columns : 
  data[feat] = climate_data[feat]

In [58]:
data.to_csv('station_data')

In [59]:
data.head()

,ID,PARAM,Value,Latitude,Longitude,Max Temp,Min Temp,Mean Temp,Heat Deg Days,Cool Deg Days,Total Precip
Date,,,,,,,,,,,
2000-01-02,08MG012,2,9.209,49.30926,-121.77373,5.3,-1.3,2.0,16.0,0.0,5.0
2000-01-03,08MG012,2,9.188,49.30926,-121.77373,2.2,-1.1,0.6,17.4,0.0,9.0
2000-01-04,08MG012,2,9.175,49.30926,-121.77373,9.1,0.9,5.0,13.0,0.0,10.8
2000-01-05,08MG012,2,9.158,49.30926,-121.77373,7.5,1.6,4.6,13.4,0.0,0.0
2000-01-06,08MG012,2,9.142,49.30926,-121.77373,3.8,0.9,2.4,15.6,0.0,7.8
